# ランダムフォレストによる特徴量選択について
HousePrices Advanced Regression Techniquesのコンペティションにおいて、特徴量を絞るためにsklearnのSelectFromModelランダムフォレストを用いた。
これについてもう少し調べてみる。

In [1]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

C:\Users\hirou\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#csvファイルの読み込みデータフレームを作成
#データはHousePrices Advanced Regression Techniquesのコンペティションより取得
train_df = pd.read_csv("train.csv", header=0)
SP=train_df[["Id","SalePrice"]]

In [3]:
#81項目ある
len(train_df.dtypes)

81

In [4]:
#欠損値の処理をする

#数値変数はmedianで穴埋め
train_df = train_df.fillna(train_df.median())

#カテゴリ変数は最大回数の値で穴埋め
null_list=list(train_df.isnull().sum()[train_df.isnull().sum()>1].index)
for i in null_list:
    max=train_df[i].value_counts()[0]
    train_df.loc[(train_df[i].isnull()),i]=max
    
#カテゴリ変数をダミー変数化
train_df=pd.get_dummies(train_df)

#特徴量とターゲットに分割
train=train_df.drop(["Id","SalePrice"],axis=1)
target=train_df["SalePrice"]

#trainの項目名のリストを作成
col=list(train.columns)

In [5]:
print("各処理後の特徴量の項目数 : ",len(train.dtypes))

各処理後の特徴量の項目数 :  303


##  確認したいこと
SelectFromModelとRandomForestRegressorを用いた項目の絞りかたが、RandomForestRegressorのfeature_importances_に基づいて選択されていることを確認する。

In [6]:
#SelectFromModelとRandomForestRegressorによって項目を絞る処理
selector = SelectFromModel(RandomForestRegressor(n_estimators=100, random_state=42), threshold="median")
selector.fit(train,target)
x_selected = selector.transform(train)
selector_ver=pd.DataFrame({"col" : col , "bool" : selector.get_support()})
selector_ver[selector_ver["bool"]==True].reset_index(drop=True)

,col,bool
0,MSSubClass,True
1,LotFrontage,True
2,LotArea,True
3,OverallQual,True
4,OverallCond,True
5,YearBuilt,True
6,YearRemodAdd,True
7,MasVnrArea,True
8,BsmtFinSF1,True
9,BsmtFinSF2,True


In [7]:
#RandomForestのfeature_importances_で項目を絞る処理
forest=RandomForestRegressor(n_estimators=100, random_state=42)
forest.fit(train,target)
imp=list(forest.feature_importances_)

imp_ver=pd.DataFrame({"col" : col,"imp" : imp})
imp_ver=imp_ver.sort_values(by="imp",ascending=False).reset_index(drop=True)

In [8]:
#feature_importances_の上から152番目(median)まで取得
imp_ver[:imp_ver[imp_ver["imp"]==imp_ver["imp"].median()].index[0]+1]

,col,imp
0,OverallQual,0.580222
1,GrLivArea,0.107616
2,TotalBsmtSF,0.036829
3,2ndFlrSF,0.032555
4,BsmtFinSF1,0.030640
5,GarageCars,0.025565
6,1stFlrSF,0.024236
7,GarageArea,0.012593
8,LotArea,0.011945
9,YearBuilt,0.008742


In [9]:
set(list(selector_ver["col"]))==set(list(imp_ver["col"]))

True

### 結論
思っていた通り、SelectFromModelをランダムフォレスト回帰で用いた場合、ランダムフォレスト回帰のfeature_importances_に基づいて選択されていることを確認することができた。

### メモ
そもそもfeature_importances_はどのように計算されているのか。  
ランダムフォレストを行うにあたり、ある項目のデータをランダムに振りなおした場合に精度がどれくらい変わるか、というのを全項目で行っている。  
x1=(2,3,4,),x2=(5,2,4),x3=(6,3,2)のようなデータがあった場合、x1=(5,3,4,),x2=(6,2,4),x3=(2,3,2)のように値を入れ替えてランダムフォレストを行い精度比較するというイメージ。これを二次元目、三次元目の値についても行ったときに、一番精度に影響があった項目が一番重要度が大きいという考え方。